In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path

print(Path().cwd())
os.chdir(Path(os.getcwd()).parent)
print(Path().cwd())

/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/CLI-Demos
/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS


## Select Contrast-Enhanced Ultrasound (CEUS) Cine and Parser

In [2]:
from src.image_loading.options import get_scan_loaders

print("Available scan loaders:", list(get_scan_loaders().keys()))

Available scan loaders: ['nifti']


In [3]:
scan_type = 'nifti'

scan_path = '/media/das/TOSHIBA EXT/P-Selectin Data/July 2019 US imaging study Mice/July 2019 batch 3 imaging day 1/July2019mouse1imagingday1/20190725103303.756.nii.gz'
scan_loader_kwargs = {
    'transpose': False,
}

In [4]:
from src.entrypoints import scan_loading_step

image_data = scan_loading_step(scan_type, scan_path, **scan_loader_kwargs)

## Load Segmentation

Assumes same segmentation for each frame

In [5]:
from src.seg_loading.options import get_seg_loaders

print("Available segmentation loaders:", list(get_seg_loaders().keys()))

Available segmentation loaders: ['nifti_voi', 'load_bolus_mask']


In [6]:
seg_type = 'nifti_voi'

seg_path = '/media/das/TOSHIBA EXT/P-Selectin Data/July VOIs (Ashley)/20190725103303.756_segmentation_LEFT.nii.gz'
seg_loader_kwargs = {}

In [7]:
from src.entrypoints import seg_loading_step

seg_data = seg_loading_step(seg_type, image_data, seg_path, scan_path, **seg_loader_kwargs)

## CEUS Quantitative Temporal Curve Analysis

In [8]:
from src.time_series_analysis.options import get_analysis_types, get_required_kwargs

all_analysis_types, all_analysis_funcs = get_analysis_types()
print("Available analysis types:", list(all_analysis_types.keys()))

Available analysis types: ['curves', 'curves_paramap']


In [9]:
analysis_type = 'curves_paramap'

print("Available analysis functions:", list(all_analysis_funcs.keys()))

Available analysis functions: ['pyradiomics', 'tic']


In [10]:
analysis_funcs = ['tic']

# Find all required kwargs for the analysis functions
analysis_funcs = analysis_funcs if len(analysis_funcs) else list(all_analysis_funcs[analysis_type].keys())
required_kwargs = get_required_kwargs(analysis_type, analysis_funcs)
print("Required kwargs for current analysis:", required_kwargs)

Required kwargs for current analysis: ['cor_vox_len', 'ax_vox_len', 'ax_vox_ovrlp', 'sag_vox_ovrlp', 'cor_vox_ovrlp', 'sag_vox_len']


In [11]:
analysis_kwargs = {
    'ax_vox_ovrlp': 50,  # %
    'sag_vox_ovrlp': 50,  # %
    'cor_vox_ovrlp': 50,  # %
    'ax_vox_len': 5.0,  # mm
    'sag_vox_len': 5.0,  # mm
    'cor_vox_len': 5.0,  # mm
    'pyradiomics_config_paths': [
        '/home/das/QuantUS_Projs/QuantUS-Plugins-CEUS/configs/pyradiomics/moderate-pselectin.yml',
    ],
    'curves_output_path': 'test2.csv',
}

In [12]:
from src.entrypoints import analysis_step

analysis_obj = analysis_step(analysis_type, image_data, seg_data, analysis_funcs, **analysis_kwargs)

Computing curves: 100%|██████████| 391/391 [00:16<00:00, 23.14it/s]


In [13]:
len(analysis_obj.curves)

72

## Curve Quantification

In [14]:
from src.curve_quantification.options import get_quantification_funcs

quantification_funcs = get_quantification_funcs()
print("Available quantification functions:", quantification_funcs.keys())

Available quantification functions: dict_keys(['auc_no_fit', 'cmus_firstorder', 'dte', 'first_order', 'lognormal_fit', 'wash_rates'])


In [15]:
function_names = ['dte'] # Empty list will use all functions
output_path = 'test_quants.csv'
curve_quantifications_kwargs = {
    'curves_to_fit': ['moderate-pselectin_diagnostics_Image-original_Mean'],
    'n_frames_to_analyze': 100,
    'tic_name': 'TIC'
}

In [16]:
from src.entrypoints import curve_quantification_step

curve_quant = curve_quantification_step(analysis_obj, function_names, output_path, **curve_quantifications_kwargs)

## Parametric Map Visualizations

In [17]:
from src.visualizations.options import get_visualization_types

types, funcs = get_visualization_types()
print("Available visualization types:", list(types.keys()))
print("Available visualization functions:", list(funcs.keys()))

Available visualization types: ['paramap']
Available visualization functions: []


In [18]:
vis_type = 'paramap'
params = [] # Empty list will use all parameters 
vis_funcs = []
vis_kwargs = {
    'paramap_folder_path': 'paramaps',
    'hide_all_visualizations': False,  # Set to True to hide all visualizations
}

In [19]:
from src.entrypoints import visualization_step

vis_obj = visualization_step(curve_quant, vis_type, params, vis_funcs, **vis_kwargs)